<a href="https://colab.research.google.com/github/NULabTMN/hw4-aidasharif1365/blob/master/CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings




In [ ]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2021-04-14 17:06:41--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.1’

shakespeare_plays.t 100%[===================>]   4.53M  10.5MB/s    in 0.4s    

2021-04-14 17:06:44 (10.5 MB/s) - ‘shakespeare_plays.txt.1’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [ ]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [ ]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8441256284713745),
 ('prince', 0.7550256252288818),
 ('clarence', 0.7451651096343994),
 ('duke', 0.7259522676467896),
 ('york', 0.7239415645599365),
 ('warwick', 0.713495671749115),
 ('cardinal', 0.7053724527359009),
 ('lady', 0.6993714570999146),
 ('margaret', 0.6955322027206421),
 ('gloucester', 0.6858654022216797)]

we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [ ]:
model.wv.similarity('othello', 'desdemona')

0.96950686

therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [ ]:
## TODO: Implement cosim
import numpy as np
from scipy.spatial import distance
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  dot=np.dot(np.array(v1),np.array(v2))
  return dot/(np.sqrt(np.dot(np.array(v1),np.array(v1))) * np.sqrt(np.dot(np.array(v2),np.array(v2))))

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.9695069

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2021-04-14 17:12:44--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M   105MB/s    in 0.6s    

2021-04-14 17:12:45 (105 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2021-04-14 17:12:45--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.1’

30k.fr.vec.1        100%[===================>]  64.66M   146MB/s    in 0.4s    

2021-04-14 17:12:45 (146 MB/s) - ‘30k.fr.vec.1’ saved [67802327/67802327]

--2021-04-14 17:12:45--  http://www.cc

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [ ]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')
arvec = ftvectors('30k.ar.vec')
esvec = ftvectors('30k.es.vec')
zhvec = ftvectors('30k.zh.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

we write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [ ]:
## implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0

  for key in vecDict:
    sim=cosim(vecDict[key], vec)
    if sim>similarity:
      similarity=sim
      mostSimilar=key
  return (mostSimilar, similarity)

## some example searches
print('French examples: ',[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']])
print('Arabic examples: ',[mostSimilar(envec[e], arvec) for e in ['aida', 'morning', 'fasting', 'religion', 'desert','dirty']])
print('Spanish examples: ',[mostSimilar(envec[e], esvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']])
print('Chinese examples: ',[mostSimilar(envec[e], zhvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']])

French examples:  [('informatique', 0.5023827767603765), ('allemagne', 0.593718413875964), ('matrice', 0.5088361302065517), ('physique', 0.4555543434796394), ('fermentation', 0.3504105196166514)]
Arabic examples:  [('عايدة', 0.2550398635325634), ('المساء', 0.4322121168738552), ('الصيام', 0.44063301000385063), ('الأديان', 0.3904924140695643), ('صحراء', 0.4056515135305318), ('مخدرات', 0.22357997287760595)]
Spanish examples:  [('computador', 0.5013697495254124), ('alemania', 0.6352798713596078), ('matriz', 0.4784864671614966), ('física', 0.4784845095690361), ('levadura', 0.5114917452709493)]
Chinese examples:  [('電腦', 0.6331072804288355), ('德國', 0.6117215949997674), ('矩陣', 0.4826503662879593), ('物理', 0.5394598078914868), ('酵母', 0.5094100865393029)]


Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2021-04-14 17:14:54--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  --.-KB/s    in 0.08s   

2021-04-14 17:14:55 (17.3 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [ ]:
words_langfr=[]
words_langes=[]
words_langzh=[]
words_langar=[]

for i in links:
  if i[1]=='fr':
    words_langfr.append(i)
  elif i[1]=='zh':
    words_langzh.append(i)
  elif i[1]=='es':
    words_langes.append(i)
  elif i[1]=='ar':
    words_langar.append(i)
print(len(words_langfr),len(words_langes),len(words_langar),len(words_langzh))

9869 8016 4102 6943


**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
Baseline= sum(words_langfr[i][0] == words_langfr[i][2] for i in range(len(words_langfr)))/len(words_langfr)
print('Baseline Accuracy in English-French is: ',Baseline)

0.6742324450298915


In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
#in this accuracy calculations the total number of words is equal to the words which are in the dictionary
sum_sim=0
total=0

for i in range(len(words_langfr)):
  if words_langfr[i][0] in envec:
    total+=1
    if mostSimilar(envec[words_langfr[i][0]], frvec)[0] == words_langfr[i][2]: 
      sum_sim+=1

5289
9869
0.5359205593271862


In [ ]:
print('Baseline Accuracy in English-French is: ',Baseline)
print('Accuracy in English-French is: ',sum_sim/total)

Baseline Accuracy in English-French is:  0.6742324450298915
Accuracy in English-French is:  0.5359205593271862


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-Spanish/Arabic/Chinese Wikipedia retrieval accuracy.
BaselineEs= sum(words_langes[i][0] == words_langes[i][2] for i in range(len(words_langes)))/len(words_langes)
BaselineAr= sum(words_langar[i][0] == words_langar[i][2] for i in range(len(words_langar)))/len(words_langar)
BaselineZh= sum(words_langzh[i][0] == words_langzh[i][2] for i in range(len(words_langzh)))/len(words_langzh)

print('Baseline Accuracy in English-Arabic is: ',BaselineAr)
print('Baseline Accuracy in English-Spanish is: ',BaselineEs)
print('Baseline Accuracy in English-Chinees is: ',BaselineZh)

Baseline Accuracy in English-Arabic is:  0.006582155046318869
Baseline Accuracy in English-Spanish is:  0.5173403193612774
Baseline Accuracy in English-Chinees is:  0.06740602045225406


In [ ]:
## TODO: Compute English-Spanish/Arabic/Chinese Wikipedia retrieval accuracy.
#in this accuracy calculations the total number of words is equal to the words which are in the dictionary
sum_simar=0
sum_simes=0
sum_simzh=0
totalar=0
totales=0
totalzh=0

for i in range(len(words_langar)):
  if words_langar[i][0] in envec:
    totalar+=1
    if mostSimilar(envec[words_langar[i][0]], frvec)[0] == words_langar[i][2]: 
      sum_simar+=1
for i in range(len(words_langes)):
  if words_langes[i][0] in envec:
    totales+=1
    if mostSimilar(envec[words_langes[i][0]], frvec)[0] == words_langes[i][2]: 
      sum_simes+=1
for i in range(len(words_langzh)):
  if words_langzh[i][0] in envec:
    totalzh+=1
    if mostSimilar(envec[words_langzh[i][0]], frvec)[0] == words_langzh[i][2]: 
      sum_simzh+=1

print('Accuracy in English-Arabic is: ',sum_simar/totalar)
print('Accuracy in English-Spanish is: ',sum_simes/totales)
print('Accuracy in English-Chineese is: ',sum_simzh/totalzh)

Accuracy in English-Arabic is:  0.005119453924914676
Accuracy in English-Spanish is:  0.2624750499001996
Accuracy in English-Chineese is:  0.03528733976667147


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".